In [1]:
from ast import Break
import re
from unittest import result
from urllib.error import URLError
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Firefox
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.events import EventFiringWebDriver, AbstractEventListener
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
import time
from time import sleep
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
url ="https://www.amazon.com/Best-Sellers-Cell-Phones/zgbs/electronics/7072561011"
products_list = []
no_of_best_selling = 50


In [3]:
driver = webdriver.Firefox()
actions = ActionChains(driver)
driver.get(url)
product_search = url.split('/')[3].replace('-', '')

In [4]:
while True:
    try:
        ActionChains(driver).send_keys(Keys.END).perform()
        sleep(1)
        products = driver.find_elements(By.CSS_SELECTOR, 'div[id="gridItemRoot"]')
        if len(products)+1 >= no_of_best_selling:
            break
    except:
        pass 

with tqdm(total=len(products),unit ="product", desc ="secondary scraper") as pbar:   
    for product in products :
        name = product.find_element(By.CSS_SELECTOR, 'div[class="_cDEzb_p13n-sc-css-line-clamp-3_g3dy1"]').text
        url = product.find_element(By.CSS_SELECTOR, 'a[class="a-link-normal"]').get_attribute('href')
        rating = float(product.find_element(By.CSS_SELECTOR, 'a > i > span[class="a-icon-alt"]').get_attribute('innerHTML').split(' ')[0])
        image = product.find_element(By.TAG_NAME, 'img').get_attribute('src')
        try:
            price = product.find_element(By.CSS_SELECTOR,'span[class="_cDEzb_p13n-sc-price_3mJ9Z"]').text
        except:
            no_of_offers = product.find_element(By.CSS_SELECTOR,'span[class="a-color-secondary"]').text
            starts_from = product.find_element(By.CSS_SELECTOR,'span[class="p13n-sc-price"]').text
            price = no_of_offers + starts_from
        
        product_dict = {'Name': name, 
                        'Product Page': url, 
                        'Image': image, 
                        'Price': price , 
                        'Rating': rating }
        
        products_list.append(product_dict)
        
        pbar.update(1) 
        
          
driver.quit()  
df = pd.DataFrame(products_list) 

secondary scraper: 100%|██████████| 50/50 [00:03<00:00, 12.60product/s]


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          50 non-null     object 
 1   Product Page  50 non-null     object 
 2   Image         50 non-null     object 
 3   Price         50 non-null     object 
 4   Rating        50 non-null     float64
dtypes: float64(1), object(4)
memory usage: 2.1+ KB


In [6]:
df.tail()

,Name,Product Page,Image,Price,Rating
45,"Samsung Galaxy S20 FE 5G (128GB, 6GB) 6.5"" AMO...",https://www.amazon.com/Samsung-Snapdragon-Resi...,https://images-na.ssl-images-amazon.com/images...,$208.00,4.1
46,"Google Pixel 4a Smartphone, 128GB Storage & Un...",https://www.amazon.com/Google-Smartphone-Stora...,https://images-na.ssl-images-amazon.com/images...,$138.00,4.0
47,LIVELY Jitterbug Smart3 Smartphone for Seniors,https://www.amazon.com/Jitterbug-Smart3-Smartp...,https://images-na.ssl-images-amazon.com/images...,3 offers from $66.00$66.00,4.0
48,"Samsung Galaxy S20 Ultra, 128GB, Cosmic Black ...",https://www.amazon.com/Samsung-Galaxy-Ultra-12...,https://images-na.ssl-images-amazon.com/images...,$291.50,4.2
49,Google Pixel 6a - 5G Android Phone - Unlocked ...,https://www.amazon.com/Google-Pixel-6a-Smartph...,https://images-na.ssl-images-amazon.com/images...,$244.00,3.8


In [7]:
df.to_csv(f'best selling {product_search} on amazon.csv')